# Solar Generation vs Cloud Cover Analysis

This notebook analyzes how cloud cover affects solar energy generation in California.

**Data Sources:**
- Solar generation: CAISO via GridStatus API
- Weather/cloud cover: NOAA Integrated Surface Database (ISD)

In [ ]:
import gridstatus
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta
import requests
import os

# Set up data directory
DATA_DIR = '../data/raw'
os.makedirs(DATA_DIR, exist_ok=True)

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
%matplotlib inline

print("✓ Setup complete")

## 1. Fetch Solar Generation Data (CAISO)

We'll use GridStatus to get solar generation data from California's grid operator.

In [ ]:
# Initialize CAISO
caiso = gridstatus.CAISO()

# Define date range (using 2024 to match available weather data)
start_date = datetime(2024, 1, 1)
end_date = datetime(2025, 1, 1)
start_str = start_date.strftime('%Y-%m-%d')
end_str = end_date.strftime('%Y-%m-%d')

# Fetch or load cached data
cache_file = f'{DATA_DIR}/solar_fuel_mix_{start_str}_{end_str}.csv'

if os.path.exists(cache_file):
    print(f"Loading from cache: {cache_file}")
    fuel_mix = pd.read_csv(cache_file, parse_dates=['Time'])
else:
    print(f"Fetching solar data from {start_str} to {end_str}...")
    fuel_mix = caiso.get_fuel_mix(start=start_str, end=end_str)
    fuel_mix.to_csv(cache_file, index=False)
    print(f"✓ Saved to cache")

# Ensure Time is datetime (handle timezone)
fuel_mix['Time'] = pd.to_datetime(fuel_mix['Time'], utc=True)

# Drop any data from 2025 (just the first timestamp)
fuel_mix = fuel_mix[fuel_mix['Time'].dt.year == 2024]

print(f"\n✓ Loaded {len(fuel_mix):,} observations")
print(f"Date range: {fuel_mix['Time'].min()} to {fuel_mix['Time'].max()}")
fuel_mix.head()

In [ ]:
# Time is already datetime, just extract date
fuel_mix['Date'] = fuel_mix['Time'].dt.date

# Calculate daily metrics
daily_solar = fuel_mix.groupby('Date').agg({
    'Solar': [
        ('Max_MW', 'max'),
        ('Mean_MW', 'mean')
    ]
}).reset_index()

# Flatten column names
daily_solar.columns = ['Date', 'Max_MW', 'Mean_MW']

# Calculate total energy (MWh) per day
daily_energy = fuel_mix.groupby('Date').agg({
    'Solar': lambda x: (x * (5/60)).sum()
}).reset_index()
daily_energy.columns = ['Date', 'Total_MWh']

# Merge
daily_solar = daily_solar.merge(daily_energy, on='Date')

# Convert Date back to datetime for plotting
daily_solar['Date'] = pd.to_datetime(daily_solar['Date'])

print(f"✓ Daily solar data calculated")
print(f"Date range: {daily_solar['Date'].min().date()} to {daily_solar['Date'].max().date()}")
print(f"Total days: {len(daily_solar)}")
print(f"\nSample data:")
print(daily_solar.head(10))

In [ ]:
import ephem
from datetime import datetime

# Sacramento coordinates (roughly central CA for CAISO)
lat, lon = 38.58, -121.49

def calculate_daylight_minutes(date, lat, lon):
    """Calculate minutes of daylight for a given date and location"""
    observer = ephem.Observer()
    observer.lat = str(lat)
    observer.lon = str(lon)
    
    # Convert pandas Timestamp to Python datetime
    if hasattr(date, 'to_pydatetime'):
        date = date.to_pydatetime()
    if date.tzinfo is not None:
        date = date.replace(tzinfo=None)
    
    # Start at midnight of this day
    midnight = datetime(date.year, date.month, date.day, 0, 0, 0)
    observer.date = midnight
    
    sun = ephem.Sun()
    
    # Get next sunrise and next sunset after midnight
    sunrise = observer.next_rising(sun).datetime()
    observer.date = sunrise  # Reset to sunrise
    sunset = observer.next_setting(sun).datetime()
    
    # Calculate daylight in minutes
    daylight = (sunset - sunrise).total_seconds() / 60
    return daylight

# Test it
test_date = pd.Timestamp('2024-06-21')
result = calculate_daylight_minutes(test_date, lat, lon)
print(f"Summer solstice {test_date.date()}: {result/60:.2f} hours")

test_date = pd.Timestamp('2024-12-21')
result = calculate_daylight_minutes(test_date, lat, lon)
print(f"Winter solstice {test_date.date()}: {result/60:.2f} hours")

# Recalculate for all days
print("\nRecalculating daylight hours for all days...")
daily_solar['Daylight_Minutes'] = daily_solar['Date'].apply(
    lambda d: calculate_daylight_minutes(d, lat, lon)
)
daily_solar['Daylight_Hours'] = daily_solar['Daylight_Minutes'] / 60

print("\nDaylight hours range:")
print(f"  Max: {daily_solar['Daylight_Hours'].max():.2f} hours")
print(f"  Min: {daily_solar['Daylight_Hours'].min():.2f} hours")
print(f"  Summer avg (Jun-Aug): {daily_solar[daily_solar['Date'].dt.month.isin([6,7,8])]['Daylight_Hours'].mean():.2f} hours")
print(f"  Winter avg (Dec-Feb): {daily_solar[daily_solar['Date'].dt.month.isin([12,1,2])]['Daylight_Hours'].mean():.2f} hours")

## 2. Visualize Solar Generation

In [ ]:
# Recreate the 3-panel plot with corrected daylight hours
fig, axes = plt.subplots(3, 1, figsize=(14, 12))

# 1. Peak MW per day
axes[0].plot(daily_solar['Date'], daily_solar['Max_MW'], linewidth=0.8, color='orange', alpha=0.8)
axes[0].set_title('Peak Solar Generation by Day (2024)', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Peak Power (MW)', fontsize=12)
axes[0].grid(True, alpha=0.3)
axes[0].set_xlim(daily_solar['Date'].min(), daily_solar['Date'].max())
axes[0].set_ylim(0, None)

# 2. Total MWh per day
axes[1].plot(daily_solar['Date'], daily_solar['Total_MWh'], linewidth=0.8, color='blue', alpha=0.8)
axes[1].set_title('Total Daily Solar Energy Generation (2024)', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Total Energy (MWh)', fontsize=12)
axes[1].grid(True, alpha=0.3)
axes[1].set_xlim(daily_solar['Date'].min(), daily_solar['Date'].max())
axes[1].set_ylim(0, None)

# 3. Daylight hours (CORRECTED)
axes[2].plot(daily_solar['Date'], daily_solar['Daylight_Hours'], linewidth=0.8, color='green', alpha=0.8)
axes[2].set_title('Daylight Hours per Day (Sacramento, 2024)', fontsize=14, fontweight='bold')
axes[2].set_ylabel('Daylight (hours)', fontsize=12)
axes[2].set_xlabel('Date', fontsize=12)
axes[2].grid(True, alpha=0.3)
axes[2].set_xlim(daily_solar['Date'].min(), daily_solar['Date'].max())
axes[2].set_ylim(0, None)

plt.tight_layout()
plt.show()

# Updated summary
print("\n" + "="*70)
print("ANNUAL SUMMARY (CORRECTED)")
print("="*70)

print(f"\nDaylight Hours:")
print(f"  Summer solstice (~June 21): {daily_solar[daily_solar['Date'].dt.month == 6]['Daylight_Hours'].max():.2f} hours")
print(f"  Winter solstice (~Dec 21): {daily_solar[daily_solar['Date'].dt.month == 12]['Daylight_Hours'].min():.2f} hours")
print(f"  Difference: {daily_solar['Daylight_Hours'].max() - daily_solar['Daylight_Hours'].min():.2f} hours")

print(f"\nSolar Generation:")
print(f"  Best day total: {daily_solar['Total_MWh'].max():,.0f} MWh on {daily_solar.loc[daily_solar['Total_MWh'].idxmax(), 'Date'].date()}")
print(f"  Worst day total: {daily_solar[daily_solar['Total_MWh'] > 1000]['Total_MWh'].min():,.0f} MWh on {daily_solar.loc[daily_solar[daily_solar['Total_MWh'] > 1000]['Total_MWh'].idxmin(), 'Date'].date()}")
print(f"  Peak power ever: {daily_solar['Max_MW'].max():,.0f} MW on {daily_solar.loc[daily_solar['Max_MW'].idxmax(), 'Date'].date()}")

# Calculate energy per daylight hour (efficiency metric)
daily_solar['MWh_per_Daylight_Hour'] = daily_solar['Total_MWh'] / daily_solar['Daylight_Hours']

print(f"\nEfficiency (MWh per daylight hour):")
print(f"  Summer avg: {daily_solar[daily_solar['Date'].dt.month.isin([6,7,8])]['MWh_per_Daylight_Hour'].mean():,.0f} MWh/hr")
print(f"  Winter avg: {daily_solar[daily_solar['Date'].dt.month.isin([12,1,2])]['MWh_per_Daylight_Hour'].mean():,.0f} MWh/hr")

In [ ]:
import numpy as np
from scipy.optimize import curve_fit

# Get top 3 days per month
def get_top_n_per_month(df, n=3):
    """Get top N days by Max_MW for each month"""
    return df.groupby([
        df['Date'].dt.year, 
        df['Date'].dt.month
    ]).apply(lambda x: x.nlargest(n, 'Max_MW')).reset_index(drop=True)

monthly_top = get_top_n_per_month(daily_solar, n=3)

print(f"Top 3 days per month: {len(monthly_top)} total days")

# Define sine wave with linear trend
def sine_with_trend(day_of_year, amplitude, phase, offset, trend):
    """Sine wave with linear trend for capacity growth"""
    seasonal = amplitude * np.sin(2 * np.pi * (day_of_year - phase) / 365)
    linear_growth = trend * day_of_year
    return seasonal + offset + linear_growth

# Convert dates to day of year for fitting
monthly_top['DayOfYear'] = monthly_top['Date'].dt.dayofyear

# Fit sine wave with trend to top days
# Initial guess: amplitude=5000, phase=172 (summer solstice), offset=15000, trend=5 MW/day
initial_guess = [5000, 172, 15000, 5]
params, covariance = curve_fit(
    sine_with_trend, 
    monthly_top['DayOfYear'], 
    monthly_top['Max_MW'],
    p0=initial_guess
)

amplitude, phase, offset, trend = params
print(f"\nFitted parameters:")
print(f"  Amplitude: {amplitude:.0f} MW (seasonal variation)")
print(f"  Phase: {phase:.0f} days (peak occurs ~day {phase:.0f})")
print(f"  Offset: {offset:.0f} MW (baseline at start of year)")
print(f"  Trend: {trend:.2f} MW/day ({trend*365:.0f} MW/year capacity growth)")

# Generate fitted curve for all days
all_days = np.arange(1, 366)
fitted_curve = sine_with_trend(all_days, amplitude, phase, offset, trend)

# Create date range for plotting
fitted_dates = pd.date_range(start='2024-01-01', end='2024-12-31', freq='D')

# Plot
fig, ax = plt.subplots(1, 1, figsize=(14, 6))

# Original daily data
ax.plot(daily_solar['Date'], daily_solar['Max_MW'], 
        linewidth=0.5, color='orange', alpha=0.3, label='Daily Peak')

# Top 3 days per month
ax.scatter(monthly_top['Date'], monthly_top['Max_MW'], 
           color='red', s=50, alpha=0.6, zorder=5, label='Top 3 Days/Month', marker='o')

# Fitted curve (sine + trend)
ax.plot(fitted_dates[:len(fitted_curve)], fitted_curve, 
        linewidth=2.5, color='darkred', linestyle='--', 
        label=f'Fitted: Sine + Linear Trend ({trend*365:.0f} MW/yr)', zorder=4)

ax.set_title('Peak Solar Generation with Seasonal Fit + Capacity Growth (2024)', 
             fontsize=14, fontweight='bold')
ax.set_ylabel('Peak Power (MW)', fontsize=12)
ax.set_xlabel('Date', fontsize=12)
ax.grid(True, alpha=0.3)
ax.set_xlim(daily_solar['Date'].min(), daily_solar['Date'].max())
ax.set_ylim(0, None)
ax.legend(loc='upper left')

plt.tight_layout()
plt.show()

# Calculate improvement over simple sine
print(f"\nCapacity growth interpretation:")
print(f"  Starting capacity (Jan 1): ~{offset:.0f} MW")
print(f"  Ending capacity (Dec 31): ~{offset + trend*365:.0f} MW")
print(f"  Total growth in 2024: ~{trend*365:.0f} MW ({(trend*365/offset)*100:.1f}% increase)")

In [ ]:
# hours of daylight, temperature, cloud cover, solar angle

In [ ]:
# Calculate theoretical energy if peak generation lasted all daylight
# Energy (MWh) = Power (MW) × Time (hours)
daily_solar['Theoretical_Max_MWh'] = daily_solar['Max_MW'] * daily_solar['Daylight_Hours']

# Calculate what percentage of this theoretical max was actually achieved
daily_solar['Capacity_Factor'] = (daily_solar['Total_MWh'] / daily_solar['Theoretical_Max_MWh']) * 100

# Plot actual vs theoretical
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# 1. Actual vs Theoretical Energy
axes[0].plot(daily_solar['Date'], daily_solar['Total_MWh'], 
             linewidth=1, color='blue', alpha=0.7, label='Actual Energy')
axes[0].plot(daily_solar['Date'], daily_solar['Theoretical_Max_MWh'], 
             linewidth=1, color='red', alpha=0.5, linestyle='--', 
             label='Theoretical Max (Peak MW × Daylight Hours)')
axes[0].set_title('Actual vs Theoretical Maximum Daily Solar Energy (2024)', 
                  fontsize=14, fontweight='bold')
axes[0].set_ylabel('Energy (MWh)', fontsize=12)
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim(0, None)
axes[0].legend()

# 2. Capacity factor (what % of theoretical max was achieved)
axes[1].plot(daily_solar['Date'], daily_solar['Capacity_Factor'], 
             linewidth=0.8, color='green', alpha=0.7)
axes[1].axhline(y=daily_solar['Capacity_Factor'].mean(), 
                color='red', linestyle='--', linewidth=2, 
                label=f'Mean: {daily_solar["Capacity_Factor"].mean():.1f}%')
axes[1].set_title('Daily Capacity Factor: Actual / Theoretical Max (%)', 
                  fontsize=14, fontweight='bold')
axes[1].set_ylabel('Capacity Factor (%)', fontsize=12)
axes[1].set_xlabel('Date', fontsize=12)
axes[1].grid(True, alpha=0.3)
axes[1].set_ylim(0, 100)
axes[1].legend()

plt.tight_layout()
plt.show()

# Statistics
print("="*70)
print("HOW WELL DOES PEAK MW × DAYLIGHT HOURS EXPLAIN DAILY ENERGY?")
print("="*70)

print(f"\nCapacity Factor (Actual / Theoretical Max):")
print(f"  Mean: {daily_solar['Capacity_Factor'].mean():.1f}%")
print(f"  Median: {daily_solar['Capacity_Factor'].median():.1f}%")
print(f"  Std Dev: {daily_solar['Capacity_Factor'].std():.1f}%")
print(f"  Range: {daily_solar['Capacity_Factor'].min():.1f}% to {daily_solar['Capacity_Factor'].max():.1f}%")

# Correlation
correlation = daily_solar['Total_MWh'].corr(daily_solar['Theoretical_Max_MWh'])
print(f"\nCorrelation between actual and theoretical: {correlation:.3f}")

# R-squared (variance explained)
from scipy.stats import linregress
slope, intercept, r_value, p_value, std_err = linregress(
    daily_solar['Theoretical_Max_MWh'], 
    daily_solar['Total_MWh']
)
r_squared = r_value ** 2
print(f"R² (variance explained): {r_squared:.3f} ({r_squared*100:.1f}%)")

print(f"\nInterpretation:")
print(f"  Peak MW × Daylight Hours explains {r_squared*100:.1f}% of daily energy variation")
print(f"  On average, actual generation is {daily_solar['Capacity_Factor'].mean():.1f}% of theoretical max")
print(f"  This makes sense because solar follows a curve (low in morning/evening, peak at noon)")

In [ ]:
Daily_Power = Capacity * Daylight_Time * Solar_Irradiance_Clear_Day * temperature_efficiency

In [ ]:
# Load NREL API key from .env
NREL_API_KEY = None

try:
    with open('../data/.env', 'r') as f:
        for line in f:
            if line.startswith('NREL_API_KEY='):
                NREL_API_KEY = line.strip().split('=')[1]
except FileNotFoundError:
    pass

if not NREL_API_KEY:
    print("⚠️  Please add NREL_API_KEY=your_key to ../data/.env")
else:
    print("✓ NREL API key loaded")
    
    # Sacramento coordinates
    lat_sac = 38.58
    lon_sac = -121.49
    
    # NSRDB download endpoint
    base_url = "https://developer.nrel.gov/api/nsrdb/v2/solar/psm3-download.csv"
    
    params = {
        'api_key': NREL_API_KEY,
        'wkt': f'POINT({lon_sac} {lat_sac})',
        'names': '2024',
        'attributes': 'ghi,dni,dhi,air_temperature,wind_speed,cloud_type',
        'leap_day': 'false',
        'utc': 'false',
        'interval': '5',  # 5-minute intervals
    }
    
    # Cache file
    nsrdb_cache = f'{DATA_DIR}/nsrdb_sacramento_2024.csv'
    
    if os.path.exists(nsrdb_cache):
        print(f"\nLoading NSRDB data from cache: {nsrdb_cache}")
        nsrdb_data = pd.read_csv(nsrdb_cache, skiprows=2)  # Skip metadata rows
    else:
        print(f"\nDownloading NSRDB data for Sacramento 2024...")
        print(f"Location: {lat_sac}, {lon_sac}")
        
        response = requests.get(base_url, params=params)
        
        if response.status_code == 200:
            # Save to cache
            with open(nsrdb_cache, 'w') as f:
                f.write(response.text)
            print(f"✓ Downloaded and saved to: {nsrdb_cache}")
            
            # Load the data (skip first 2 rows which are metadata)
            nsrdb_data = pd.read_csv(nsrdb_cache, skiprows=2)
        else:
            print(f"❌ Error: {response.status_code}")
            print(response.text)
            nsrdb_data = None
    
    if nsrdb_data is not None:
        print(f"\n✓ NSRDB data loaded")
        print(f"Shape: {nsrdb_data.shape}")
        print(f"\nColumns: {nsrdb_data.columns.tolist()}")
        print(f"\nFirst few rows:")
        print(nsrdb_data.head())

In [ ]:
# Load the NSRDB data file you already downloaded
nsrdb_file = '/Users/lucas/Downloads/fb634fb0a8b457b3fe2534f1280f531c/87370_38.58_-121.49_2024.csv'

# NSRDB files have 2 header rows with metadata, then column names on row 3
nsrdb_data = pd.read_csv(nsrdb_file, skiprows=2)

print(f"✓ NSRDB data loaded from downloaded file")
print(f"Shape: {nsrdb_data.shape}")
print(f"\nColumns: {nsrdb_data.columns.tolist()}")
print(f"\nFirst few rows:")
print(nsrdb_data.head())

# Create datetime index
nsrdb_data['DateTime'] = pd.to_datetime(
    nsrdb_data[['Year', 'Month', 'Day', 'Hour', 'Minute']]
)
nsrdb_data['Date'] = nsrdb_data['DateTime'].dt.date

# Show summary stats for key solar radiation columns
print("\n📊 Solar radiation summary:")
for col in ['GHI', 'DNI', 'DHI']:
    if col in nsrdb_data.columns:
        print(f"{col}: min={nsrdb_data[col].min():.1f}, max={nsrdb_data[col].max():.1f}, mean={nsrdb_data[col].mean():.1f}")

In [ ]:
# Daily TOTAL irradiation (energy)
# NSRDB is 5-minute intervals, so multiply by (5/60) hours to get Wh/m²
daily_nsrdb_energy = nsrdb_data.groupby('Date').agg({
    'GHI': lambda x: (x * (5/60)).sum(),           # Total Wh/m² per day
    'Clearsky GHI': lambda x: (x * (5/60)).sum(),
    'DNI': lambda x: (x * (5/60)).sum(),
    'Clearsky DNI': lambda x: (x * (5/60)).sum(),
    'DHI': lambda x: (x * (5/60)).sum(),
    'Clearsky DHI': lambda x: (x * (5/60)).sum(),
    'Solar Zenith Angle': 'mean',
    'Cloud Type': 'mean',
    'Temperature': 'mean'
}).reset_index()

daily_nsrdb_energy.columns = ['Date', 'Total_GHI', 'Total_Clearsky_GHI', 
                               'Total_DNI', 'Total_Clearsky_DNI',
                               'Total_DHI', 'Total_Clearsky_DHI',
                               'Avg_Zenith_Angle', 'Avg_Cloud_Type', 'Avg_Temp']

daily_nsrdb_energy['Date'] = pd.to_datetime(daily_nsrdb_energy['Date'])

# Calculate total tracking irradiation
daily_nsrdb_energy['Total_Tracking_Irradiation'] = daily_nsrdb_energy['Total_DNI'] + daily_nsrdb_energy['Total_DHI']
daily_nsrdb_energy['Total_Clearsky_Tracking'] = daily_nsrdb_energy['Total_Clearsky_DNI'] + daily_nsrdb_energy['Total_Clearsky_DHI']

# Merge with CAISO data
solar_combined_energy = daily_solar.merge(daily_nsrdb_energy, on='Date', how='inner')

print("📊 Correlation with Total MWh (using daily energy totals):")
print(f"Total GHI (Fixed) vs Total MWh:               {solar_combined_energy['Total_GHI'].corr(solar_combined_energy['Total_MWh']):.4f}")
print(f"Total DNI+DHI (Tracking) vs Total MWh:        {solar_combined_energy['Total_Tracking_Irradiation'].corr(solar_combined_energy['Total_MWh']):.4f}")
print(f"Total Clearsky Tracking vs Total MWh:         {solar_combined_energy['Total_Clearsky_Tracking'].corr(solar_combined_energy['Total_MWh']):.4f}")

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(6, 1, figsize=(14, 16))

# Plot 1: Total GHI - Actual vs Clearsky
axes[0].plot(solar_combined_energy['Date'], solar_combined_energy['Total_GHI'], 
             label='Actual Total GHI (Fixed Panels)', alpha=0.7, linewidth=1)
axes[0].plot(solar_combined_energy['Date'], solar_combined_energy['Total_Clearsky_GHI'], 
             label='Clearsky Total GHI', alpha=0.7, linewidth=1, linestyle='--')
axes[0].set_ylabel('Total GHI (Wh/m²)')
axes[0].set_title('Daily Total Global Horizontal Irradiation (Fixed Panels)')
axes[0].legend()
axes[0].set_ylim(bottom=0)
axes[0].grid(True, alpha=0.3)

# Plot 2: Total DNI+DHI - Actual vs Clearsky
axes[1].plot(solar_combined_energy['Date'], solar_combined_energy['Total_Tracking_Irradiation'], 
             label='Actual Total DNI+DHI (Tracking Panels)', alpha=0.7, linewidth=1, color='green')
axes[1].plot(solar_combined_energy['Date'], solar_combined_energy['Total_Clearsky_Tracking'], 
             label='Clearsky Total DNI+DHI', alpha=0.7, linewidth=1, linestyle='--', color='lightgreen')
axes[1].set_ylabel('Total DNI+DHI (Wh/m²)')
axes[1].set_title('Daily Total Tracking Panel Irradiation (DNI + DHI)')
axes[1].legend()
axes[1].set_ylim(bottom=0)
axes[1].grid(True, alpha=0.3)

# Plot 3: CAISO Total MWh
axes[2].plot(solar_combined_energy['Date'], solar_combined_energy['Total_MWh'], 
             label='Total Daily Energy', alpha=0.7, linewidth=1, color='orange')
axes[2].set_ylabel('Energy (MWh)')
axes[2].set_title('CAISO Solar Total Daily Energy Production')
axes[2].legend()
axes[2].set_ylim(bottom=0)
axes[2].grid(True, alpha=0.3)

# Plot 4: Total GHI vs Total MWh (Fixed Panels)
ax4a = axes[3]
ax4b = ax4a.twinx()
ax4a.plot(solar_combined_energy['Date'], solar_combined_energy['Total_GHI'], 
          label='Total GHI (Fixed)', alpha=0.7, linewidth=1, color='blue')
ax4b.plot(solar_combined_energy['Date'], solar_combined_energy['Total_MWh'], 
          label='Total MWh', alpha=0.7, linewidth=1, color='orange')
ax4a.set_ylabel('Total GHI (Wh/m²)', color='blue')
ax4b.set_ylabel('Energy (MWh)', color='orange')
ax4a.set_title(f'Fixed Panels: Total GHI vs Total Energy (correlation: {solar_combined_energy["Total_GHI"].corr(solar_combined_energy["Total_MWh"]):.3f})')
ax4a.tick_params(axis='y', labelcolor='blue')
ax4b.tick_params(axis='y', labelcolor='orange')
ax4a.set_ylim(bottom=0)
ax4b.set_ylim(bottom=0)
ax4a.grid(True, alpha=0.3)

# Plot 5: Total DNI+DHI vs Total MWh (Tracking Panels)
ax5a = axes[4]
ax5b = ax5a.twinx()
ax5a.plot(solar_combined_energy['Date'], solar_combined_energy['Total_Tracking_Irradiation'], 
          label='Total DNI+DHI (Tracking)', alpha=0.7, linewidth=1, color='green')
ax5b.plot(solar_combined_energy['Date'], solar_combined_energy['Total_MWh'], 
          label='Total MWh', alpha=0.7, linewidth=1, color='orange')
ax5a.set_ylabel('Total DNI+DHI (Wh/m²)', color='green')
ax5b.set_ylabel('Energy (MWh)', color='orange')
ax5a.set_title(f'Tracking Panels: Total DNI+DHI vs Total Energy (correlation: {solar_combined_energy["Total_Tracking_Irradiation"].corr(solar_combined_energy["Total_MWh"]):.3f})')
ax5a.tick_params(axis='y', labelcolor='green')
ax5b.tick_params(axis='y', labelcolor='orange')
ax5a.set_ylim(bottom=0)
ax5b.set_ylim(bottom=0)
ax5a.grid(True, alpha=0.3)

# Plot 6: Clearsky Total DNI+DHI vs Total MWh (Theoretical Maximum for Tracking)
ax6a = axes[5]
ax6b = ax6a.twinx()
ax6a.plot(solar_combined_energy['Date'], solar_combined_energy['Total_Clearsky_Tracking'], 
          label='Clearsky Total DNI+DHI', alpha=0.7, linewidth=1, color='purple')
ax6b.plot(solar_combined_energy['Date'], solar_combined_energy['Total_MWh'], 
          label='Total MWh', alpha=0.7, linewidth=1, color='orange')
ax6a.set_ylabel('Clearsky Total DNI+DHI (Wh/m²)', color='purple')
ax6b.set_ylabel('Energy (MWh)', color='orange')
ax6a.set_title(f'Clearsky Tracking: Total Clearsky DNI+DHI vs Total Energy (correlation: {solar_combined_energy["Total_Clearsky_Tracking"].corr(solar_combined_energy["Total_MWh"]):.3f})')
ax6a.tick_params(axis='y', labelcolor='purple')
ax6b.tick_params(axis='y', labelcolor='orange')
ax6a.set_ylim(bottom=0)
ax6b.set_ylim(bottom=0)
ax6a.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print detailed correlation comparison
print("\n📊 Correlation Comparison (Daily Total Energy):")
print(f"Total GHI (Fixed Panels) vs Total MWh:              {solar_combined_energy['Total_GHI'].corr(solar_combined_energy['Total_MWh']):.4f}")
print(f"Total DNI+DHI (Tracking Panels) vs Total MWh:       {solar_combined_energy['Total_Tracking_Irradiation'].corr(solar_combined_energy['Total_MWh']):.4f}")
print(f"Total Clearsky DNI+DHI (Theoretical Max) vs Total MWh: {solar_combined_energy['Total_Clearsky_Tracking'].corr(solar_combined_energy['Total_MWh']):.4f}")

In [ ]:
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

# Define sine wave with linear trend
def sine_with_trend(day_of_year, amplitude, phase, offset, trend):
    seasonal = amplitude * np.sin(2 * np.pi * (day_of_year - phase) / 365)
    linear_growth = trend * day_of_year
    return seasonal + offset + linear_growth

# Add day of year column
solar_combined_energy['DayOfYear'] = solar_combined_energy['Date'].dt.dayofyear

# Fit curve to actual Total DNI+DHI
params_actual, _ = curve_fit(
    sine_with_trend, 
    solar_combined_energy['DayOfYear'], 
    solar_combined_energy['Total_Tracking_Irradiation'], 
    p0=[2000, 172, 6000, 0]
)

# Fit curve to clearsky Total DNI+DHI
params_clearsky, _ = curve_fit(
    sine_with_trend, 
    solar_combined_energy['DayOfYear'], 
    solar_combined_energy['Total_Clearsky_Tracking'], 
    p0=[2000, 172, 8000, 0]
)

# Generate fitted curves for 366 days (2024 is a leap year)
days = np.arange(1, 367)
fitted_actual = sine_with_trend(days, *params_actual)
fitted_clearsky = sine_with_trend(days, *params_clearsky)

# Create date range for plotting fitted curves (366 days)
dates_fitted = pd.date_range('2024-01-01', '2024-12-31', freq='D')

# Define solstice dates
summer_solstice = pd.Timestamp('2024-06-21')
winter_solstice = pd.Timestamp('2024-12-21')

# Get data for solstices
summer_data = solar_combined_energy[solar_combined_energy['Date'] == summer_solstice]
winter_data = solar_combined_energy[solar_combined_energy['Date'] == winter_solstice]

print(f"Actual DNI+DHI fit: amplitude={params_actual[0]:.1f}, phase={params_actual[1]:.1f}, offset={params_actual[2]:.1f}, trend={params_actual[3]:.2f}")
print(f"Clearsky DNI+DHI fit: amplitude={params_clearsky[0]:.1f}, phase={params_clearsky[1]:.1f}, offset={params_clearsky[2]:.1f}, trend={params_clearsky[3]:.2f}")

# Create plots
fig, axes = plt.subplots(6, 1, figsize=(14, 16))

# Plot 1: Total GHI - Actual vs Clearsky
axes[0].plot(solar_combined_energy['Date'], solar_combined_energy['Total_GHI'], 
             label='Actual Total GHI (Fixed Panels)', alpha=0.7, linewidth=1)
axes[0].plot(solar_combined_energy['Date'], solar_combined_energy['Total_Clearsky_GHI'], 
             label='Clearsky Total GHI', alpha=0.7, linewidth=1, linestyle='--')
axes[0].scatter([summer_solstice, winter_solstice], 
                [summer_data['Total_GHI'].values[0], winter_data['Total_GHI'].values[0]], 
                color='red', s=100, zorder=5, label='Solstices')
axes[0].set_ylabel('Total GHI (Wh/m²)')
axes[0].set_title('Daily Total Global Horizontal Irradiation (Fixed Panels)')
axes[0].legend()
axes[0].set_ylim(bottom=0)
axes[0].grid(True, alpha=0.3)

# Plot 2: Total DNI+DHI - Actual vs Clearsky WITH FITTED CURVES
axes[1].plot(solar_combined_energy['Date'], solar_combined_energy['Total_Tracking_Irradiation'], 
             label='Actual Total DNI+DHI', alpha=0.5, linewidth=1, color='green')
axes[1].plot(solar_combined_energy['Date'], solar_combined_energy['Total_Clearsky_Tracking'], 
             label='Clearsky Total DNI+DHI', alpha=0.5, linewidth=1, linestyle='--', color='lightgreen')
axes[1].plot(dates_fitted, fitted_actual, 
             label='Fitted Actual DNI+DHI', linewidth=2, color='darkgreen')
axes[1].plot(dates_fitted, fitted_clearsky, 
             label='Fitted Clearsky DNI+DHI', linewidth=2, linestyle='--', color='lime')
axes[1].scatter([summer_solstice, winter_solstice], 
                [summer_data['Total_Tracking_Irradiation'].values[0], winter_data['Total_Tracking_Irradiation'].values[0]], 
                color='red', s=100, zorder=5, label='Solstices')
axes[1].set_ylabel('Total DNI+DHI (Wh/m²)')
axes[1].set_title('Daily Total Tracking Panel Irradiation (DNI + DHI) with Fitted Curves')
axes[1].legend()
axes[1].set_ylim(bottom=0)
axes[1].grid(True, alpha=0.3)

# Plot 3: CAISO Total MWh
axes[2].plot(solar_combined_energy['Date'], solar_combined_energy['Total_MWh'], 
             label='Total Daily Energy', alpha=0.7, linewidth=1, color='orange')
axes[2].scatter([summer_solstice, winter_solstice], 
                [summer_data['Total_MWh'].values[0], winter_data['Total_MWh'].values[0]], 
                color='red', s=100, zorder=5, label='Solstices')
axes[2].set_ylabel('Energy (MWh)')
axes[2].set_title('CAISO Solar Total Daily Energy Production')
axes[2].legend()
axes[2].set_ylim(bottom=0)
axes[2].grid(True, alpha=0.3)

# Plot 4: Total GHI vs Total MWh (Fixed Panels)
ax4a = axes[3]
ax4b = ax4a.twinx()
ax4a.plot(solar_combined_energy['Date'], solar_combined_energy['Total_GHI'], 
          label='Total GHI (Fixed)', alpha=0.7, linewidth=1, color='blue')
ax4b.plot(solar_combined_energy['Date'], solar_combined_energy['Total_MWh'], 
          label='Total MWh', alpha=0.7, linewidth=1, color='orange')
ax4a.scatter([summer_solstice, winter_solstice], 
             [summer_data['Total_GHI'].values[0], winter_data['Total_GHI'].values[0]], 
             color='red', s=100, zorder=5)
ax4a.set_ylabel('Total GHI (Wh/m²)', color='blue')
ax4b.set_ylabel('Energy (MWh)', color='orange')
ax4a.set_title(f'Fixed Panels: Total GHI vs Total Energy (correlation: {solar_combined_energy["Total_GHI"].corr(solar_combined_energy["Total_MWh"]):.3f})')
ax4a.tick_params(axis='y', labelcolor='blue')
ax4b.tick_params(axis='y', labelcolor='orange')
ax4a.set_ylim(bottom=0)
ax4b.set_ylim(bottom=0)
ax4a.grid(True, alpha=0.3)

# Plot 5: Total DNI+DHI vs Total MWh (Tracking Panels)
ax5a = axes[4]
ax5b = ax5a.twinx()
ax5a.plot(solar_combined_energy['Date'], solar_combined_energy['Total_Tracking_Irradiation'], 
          label='Total DNI+DHI (Tracking)', alpha=0.7, linewidth=1, color='green')
ax5b.plot(solar_combined_energy['Date'], solar_combined_energy['Total_MWh'], 
          label='Total MWh', alpha=0.7, linewidth=1, color='orange')
ax5a.scatter([summer_solstice, winter_solstice], 
             [summer_data['Total_Tracking_Irradiation'].values[0], winter_data['Total_Tracking_Irradiation'].values[0]], 
             color='red', s=100, zorder=5)
ax5a.set_ylabel('Total DNI+DHI (Wh/m²)', color='green')
ax5b.set_ylabel('Energy (MWh)', color='orange')
ax5a.set_title(f'Tracking Panels: Total DNI+DHI vs Total Energy (correlation: {solar_combined_energy["Total_Tracking_Irradiation"].corr(solar_combined_energy["Total_MWh"]):.3f})')
ax5a.tick_params(axis='y', labelcolor='green')
ax5b.tick_params(axis='y', labelcolor='orange')
ax5a.set_ylim(bottom=0)
ax5b.set_ylim(bottom=0)
ax5a.grid(True, alpha=0.3)

# Plot 6: Clearsky Total DNI+DHI vs Total MWh (Theoretical Maximum for Tracking)
ax6a = axes[5]
ax6b = ax6a.twinx()
ax6a.plot(solar_combined_energy['Date'], solar_combined_energy['Total_Clearsky_Tracking'], 
          label='Clearsky Total DNI+DHI', alpha=0.7, linewidth=1, color='purple')
ax6b.plot(solar_combined_energy['Date'], solar_combined_energy['Total_MWh'], 
          label='Total MWh', alpha=0.7, linewidth=1, color='orange')
ax6a.scatter([summer_solstice, winter_solstice], 
             [summer_data['Total_Clearsky_Tracking'].values[0], winter_data['Total_Clearsky_Tracking'].values[0]], 
             color='red', s=100, zorder=5)
ax6a.set_ylabel('Clearsky Total DNI+DHI (Wh/m²)', color='purple')
ax6b.set_ylabel('Energy (MWh)', color='orange')
ax6a.set_title(f'Clearsky Tracking: Total Clearsky DNI+DHI vs Total Energy (correlation: {solar_combined_energy["Total_Clearsky_Tracking"].corr(solar_combined_energy["Total_MWh"]):.3f})')
ax6a.tick_params(axis='y', labelcolor='purple')
ax6b.tick_params(axis='y', labelcolor='orange')
ax6a.set_ylim(bottom=0)
ax6b.set_ylim(bottom=0)
ax6a.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Pick summer and winter days
summer_day = '2024-06-21'
winter_day = '2024-12-21'

# Filter CAISO data for those days
summer_caiso = fuel_mix[fuel_mix['Time'].dt.date == pd.to_datetime(summer_day).date()].copy()
winter_caiso = fuel_mix[fuel_mix['Time'].dt.date == pd.to_datetime(winter_day).date()].copy()

# Convert to Pacific Time
summer_caiso['Time_Local'] = summer_caiso['Time'].dt.tz_convert('America/Los_Angeles')
winter_caiso['Time_Local'] = winter_caiso['Time'].dt.tz_convert('America/Los_Angeles')

# Create hour of day for easier comparison
summer_caiso['Hour'] = summer_caiso['Time_Local'].dt.hour + summer_caiso['Time_Local'].dt.minute / 60
winter_caiso['Hour'] = winter_caiso['Time_Local'].dt.hour + winter_caiso['Time_Local'].dt.minute / 60

# Plot
fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(summer_caiso['Hour'], summer_caiso['Solar'], 
        label=f'Summer ({summer_day})', linewidth=2, color='orange', marker='o', markersize=3)
ax.plot(winter_caiso['Hour'], winter_caiso['Solar'], 
        label=f'Winter ({winter_day})', linewidth=2, color='blue', marker='o', markersize=3)

ax.set_xlabel('Hour of Day (Pacific Time)')
ax.set_ylabel('Solar Generation (MW)')
ax.set_title('CAISO Solar Generation: Summer vs Winter Day')
ax.legend()
ax.set_ylim(bottom=0)
ax.grid(True, alpha=0.3)
ax.set_xlim(0, 24)

plt.tight_layout()
plt.show()

# Print summary stats
print(f"\n📊 Summary for {summer_day}:")
print(f"Peak: {summer_caiso['Solar'].max():.0f} MW at {summer_caiso.loc[summer_caiso['Solar'].idxmax(), 'Time_Local'].strftime('%I:%M %p %Z')}")
print(f"Total: {(summer_caiso['Solar'] * (5/60)).sum():.0f} MWh")

print(f"\n📊 Summary for {winter_day}:")
print(f"Peak: {winter_caiso['Solar'].max():.0f} MW at {winter_caiso.loc[winter_caiso['Solar'].idxmax(), 'Time_Local'].strftime('%I:%M %p %Z')}")
print(f"Total: {(winter_caiso['Solar'] * (5/60)).sum():.0f} MWh")

print(f"\n📊 Summer vs Winter:")
print(f"Peak ratio (summer/winter): {summer_caiso['Solar'].max() / winter_caiso['Solar'].max():.2f}x")
print(f"Total ratio (summer/winter): {(summer_caiso['Solar'] * (5/60)).sum() / (winter_caiso['Solar'] * (5/60)).sum():.2f}x")